In [1]:
import cv2
import pandas as pd
import numpy as np
import os
from detector import MediaPipeProcessor
from config import Config
# ---------- Khởi tạo model ----------
model = MediaPipeProcessor(Config())
vid_not_kept = []
df_video = pd.DataFrame()

2025-06-22 02:38:15.105528: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-22 02:38:15.115521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750534695.127375   22899 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750534695.130822   22899 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-22 02:38:15.142937: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
df = pd.read_csv('top50_daily.csv')

In [3]:
def moved(curr_kps: np.ndarray, ref_kps: np.ndarray, threshold=0.36) -> bool:
    """
    Trả về True nếu có ít nhất một phần tử của curr_kps khác ref_kps
    hơn ngưỡng threshold (normalized).
    """
    # So sánh độ chênh tuyệt đối từng chiều
    diffs = np.abs(curr_kps - ref_kps)
    return np.any(diffs > threshold)

In [4]:
def filter_video(id, src):
    cap = cv2.VideoCapture(src)
    # 1. Đọc khung đầu tiên, lấy keypoints làm reference
    
    ret, frame0 = cap.read()
    if not ret:
        print(f"✘ Không đọc được khung đầu của {src}")
        cap.release()
        return

    image0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2RGB)
    _, ref_res = model.process_frame(image0)
    ref_kps = model.extract_keypoints(ref_res)
    label = [0]
    kpts = [ref_kps]
    frame = 1
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, res = model.process_frame(image_rgb)
        if res.pose_landmarks:
            kpts_extract = model.extract_keypoints(res)
            if moved(kpts_extract, ref_kps):
                label.append(-1)
            else:
                label.append(0)
            kpts.append(kpts_extract)
    label = np.array(label)
    indices = np.where(label == -1)[0]
    if len(indices) > 0:
        last_idx = indices[-1]
        needed = 5

        # Số phần tử còn lại từ vị trí -1 đến hết
        after = len(label) - last_idx

        if after >= needed:
            label[last_idx : last_idx + needed] = id
        else:
            extra = needed - after
            start_idx = max(0, last_idx - extra)
            label[start_idx:] = id
    else:
        label[-5:] = id
    #all element in label += 1
    label += 1
    kpts = np.array(kpts)
    return kpts, label

In [5]:
video_data = []
files_name = 'top50_daily.csv'
df = pd.read_csv(files_name)
for i, row in df.iterrows():
    files_name = row['VIDEO'].split(', ')
    id = row['STT']
    for file_name in files_name:
        file_name_output = file_name.split('.')[0]
        src = os.path.join('Dataset/Video', file_name)
        dst_kpts = os.path.join('data/Keypoints_TCN', file_name_output)
        dst_label = os.path.join('data/Labels', file_name_output)
        if not os.path.exists(src):
            print(f"✘ Không tìm thấy video: {src}")
            continue
        res_kpts, res_label = filter_video(id, src)
        res_kpts = res_kpts.reshape(res_kpts.shape[0], -1)
        #save keypoints and labels
        if res_kpts is None or res_label is None:
            print(f"✘ Không xử lý được video: {src}")
            continue
        np.save(dst_kpts, res_kpts)
        np.save(dst_label, res_label)
        print(f'{id} - {file_name} - {res_kpts.shape} frames - {res_label.shape} labels')

1 - W03373B.mp4 - (127, 150) frames - (127,) labels
1 - W03373N.mp4 - (80, 150) frames - (80,) labels
1 - W03373T.mp4 - (127, 150) frames - (127,) labels
1 - W03450B.mp4 - (102, 150) frames - (102,) labels
1 - W03450T.mp4 - (114, 150) frames - (114,) labels
1 - W03450N.mp4 - (114, 150) frames - (114,) labels
2 - W01122B.mp4 - (67, 150) frames - (67,) labels
2 - W01122T.mp4 - (88, 150) frames - (88,) labels
2 - W01122N.mp4 - (88, 150) frames - (88,) labels
2 - W01122.mp4 - (161, 150) frames - (161,) labels
3 - W03175N.mp4 - (124, 150) frames - (124,) labels
3 - W03175T.mp4 - (123, 150) frames - (123,) labels
3 - W03175B.mp4 - (123, 150) frames - (123,) labels
3 - W03175.mp4 - (81, 150) frames - (81,) labels
4 - D0156N.mp4 - (139, 150) frames - (139,) labels
4 - D0156B.mp4 - (110, 150) frames - (110,) labels
4 - D0156.mp4 - (163, 150) frames - (163,) labels
4 - D0156T.mp4 - (139, 150) frames - (139,) labels
5 - W03732T.mp4 - (106, 150) frames - (106,) labels
5 - W03732.mp4 - (148, 150) f

In [6]:
df_video.head()

""
